In [4]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://instance-1.us-west1-b.c.forward-ace-411913.internal:7077") \
    .appName('spark_homework') \
    .getOrCreate()


24/03/30 20:29:13 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [96]:
spark.version

'3.3.2'

In [5]:
 import pandas as pd

In [29]:
 pdf= pd.read_csv('data/raw/fhv/fhv_tripdata_2019-10.csv.gz',compression='gzip' , nrows=1000)
#pdf.iteritems =pdf.items   

In [36]:
pdf

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014
...,...,...,...,...,...,...,...
995,B00111,2019-10-01 01:54:57,2019-10-01 03:10:06,264.0,100.0,NaN,B00111
996,B00111,2019-10-01 01:07:06,2019-10-01 01:29:54,264.0,188.0,NaN,B00111
997,B00112,2019-10-01 01:05:36,2019-10-01 01:36:28,264.0,228.0,NaN,B00112
998,B00160,2019-10-01 01:20:00,2019-10-01 01:26:00,264.0,264.0,NaN,B00160


In [31]:
pdf.Affiliated_base_number = pdf.Affiliated_base_number.astype(str)

In [32]:
spark.createDataFrame(pdf).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [39]:
from pyspark.sql import types
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True)
    , types.StructField('pickup_datetime', types.TimestampType(), True)
    , types.StructField('dropOff_datetime', types.TimestampType(), True)
    , types.StructField('PUlocationID', types.IntegerType(), True)
    , types.StructField('DOlocationID', types.IntegerType(), True)
    , types.StructField('SR_Flag', types.DoubleType(), True)
    , types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [40]:
df= spark.read \
    .option("header","True") \
    .schema(schema=schema) \
    .csv('data/raw/fhv/fhv_tripdata_2019-10.csv.gz')

In [41]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [43]:
df \
    .repartition(6) \
    .write \
    .parquet('data/pq/fhv/2019/10',mode='overwrite')

In [44]:
rdf = spark.read.parquet("data/pq/fhv/2019/10/*")

In [63]:
rdf.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [45]:
rdf.createOrReplaceTempView('fhv_data')

In [47]:
trips_15_oct = spark.sql("""

select count(*) cnt  from fhv_data where cast(pickup_datetime as date) = '2019-10-15'
""")

In [50]:
trips_15_oct.show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [64]:
SELECT 
    dropOff_datetime, 
    pickup_datetime,
    HOUR(FROM_UNIXTIME(UNIX_TIMESTAMP(dropOff_datetime) - UNIX_TIMESTAMP(pickup_datetime))) AS hour_difference
FROM 
    fhv_data 
LIMIT 10

IndentationError: unexpected indent (1053346520.py, line 2)

In [73]:
longest_hour = spark.sql("""

select dropOff_datetime, pickup_datetime,
(dropOff_datetime   - pickup_datetime)* 24  AS hour_difference
from fhv_data order by 3 desc  limit 1
""")

In [74]:
longest_hour.show()

+-------------------+-------------------+--------------------+
|   dropOff_datetime|    pickup_datetime|     hour_difference|
+-------------------+-------------------+--------------------+
|2091-10-11 18:30:00|2019-10-11 18:00:00|INTERVAL '631152 ...|
+-------------------+-------------------+--------------------+



In [77]:
zones_df = spark.read.csv('data/raw/zones/taxi_zone_lookup.csv', header=True,)

In [78]:
zones_df.createOrReplaceTempView('zones')

In [81]:
zones_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [94]:
least_pickup_zone = spark.sql("""

select Zone,count(*)  
from fhv_data a
left outer join zones  b on a.PUlocationID = b.LocationID 
group by Zone order by 2
""")

In [95]:
least_pickup_zone.show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+
only showing top 20 rows

